In [1]:
!pip install mido

In [0]:
# import keras modules 
from tensorflow.keras import models # model module 
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout # layers for model building 
from tensorflow.keras.models import Sequential # sequential class for models 
from tensorflow.keras.callbacks import TensorBoard # for viz of graph and model params 
from tensorflow.keras.preprocessing import sequence # for sequencing 
from tensorflow.keras.callbacks import ModelCheckpoint # to create check points for model 

# helper libraries 
import numpy as np # for matrix maths 

import mido # for music ops 
from mido import MidiFile, MidiTrack, Message # for music file handeling 

import pickle # for data unpacking

import h5py # to save file in h5py format

In [3]:
# import data 
with open('./notes.pkl','rb') as f:
    notes = pickle.load(f)

# first 5 
print(notes[:5])

[[60, 102, 0.0], [64, 81, 0.0], [67, 81, 0.0], [60, 81, 0.0], [48, 81, 0.0]]


In [0]:
# scale data set 
t = [] # empty list to store time 
for note in notes:
    note[0] = (note[0] - 24) / 88 
    note[1] = (note[1]) / 127
    t.append(note[2])
max_t = max(t) # max time 
for note in notes:
    note[2] /= max_t

In [5]:
# create data and labels for prediction 
X = [] # empty list for features
Y = [] # empty list for predictions

n_prev = 50 # num of notes to predict

for i in range(len(notes) - n_prev):
    x = notes[i:i+n_prev]
    y = notes[i+n_prev]
    
    X.append(x)
    Y.append(y)
    
# convert them into arrays
X = np.array(X)
Y = np.array(Y)

# print values of x and y 
print("Size of X {}".format(X.shape))
print("Size of Y {}".format(Y.shape))

Size of X (18229, 50, 3)
Size of Y (18229, 3)


In [6]:
# build a keras model 
print("Building model......")
model = Sequential() # initialize a sequential model 
model.add(LSTM(512,return_sequences=False, input_shape=(n_prev, 3))) # 2nd layer of LSTM 
model.add(Dropout(0.75)) # 2nd layer of Dropout
model.add(Dense(3)) # a dense layer of 3 tensors 
model.add(Activation('linear'))


# compile model 
print("Compiling model.....")
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
print("Model summary......")
model.summary()

Building model......
Compiling model.....
Model summary......
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               1056768   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 1539      
_________________________________________________________________
activation (Activation)      (None, 3)                 0         
Total params: 1,058,307
Trainable params: 1,058,307
Non-trainable params: 0
_________________________________________________________________


In [0]:
# make a callback for tensorboard viz 
tensorboard = TensorBoard(log_dir='./logs/music_research', histogram_freq=1)

In [9]:
# fit model 
model.fit(X, Y, batch_size=200, epochs=200, callbacks=[tensorboard], validation_split=0.2)

Train on 14583 samples, validate on 3646 samples
Epoch 1/200
14583/14583 [==============================] - 17s 1ms/step - loss: 0.0286 - acc: 0.7156 - val_loss: 0.0145 - val_acc: 0.8922
Epoch 2/200
14583/14583 [==============================] - 16s 1ms/step - loss: 0.0206 - acc: 0.7474 - val_loss: 0.0131 - val_acc: 0.8914
Epoch 3/200
14583/14583 [==============================] - 16s 1ms/step - loss: 0.0181 - acc: 0.7733 - val_loss: 0.0123 - val_acc: 0.8950
Epoch 4/200
14583/14583 [==============================] - 16s 1ms/step - loss: 0.0164 - acc: 0.8046 - val_loss: 0.0136 - val_acc: 0.9037
Epoch 5/200
14583/14583 [==============================] - 16s 1ms/step - loss: 0.0154 - acc: 0.8348 - val_loss: 0.0133 - val_acc: 0.9144
Epoch 6/200
14583/14583 [==============================] - 16s 1ms/step - loss: 0.0146 - acc: 0.8497 - val_loss: 0.0131 - val_acc: 0.9194
Epoch 7/200
14583/14583 [==============================] - 16s 1ms/step - loss: 0.0140 - acc: 0.8639 - val_loss: 0.0109 - v

In [0]:
# saving model 
model.save("music_model_mine200.h5")

In [0]:
model_string = model.to_json()

with open("music_model_mine200.json","w") as f:
  f.write(model_string)